# Détecter une fraude du service des achats à partir de la Data Analyse

Vous souhaitez auditer le service des achats d’une entreprise dont vous avez eu vent d’irrégularité sur les dépenses pétrolières. Vous avez pour cela intégré les services SAP qui ont structuré vos données selon leur nomenclature.


**Votre rôle** : identifier l’irrégularité et la personne à l’origine.

In [0]:
# A exécuter (Alt + Entrée)
%pip install vega

In [0]:
# A exécuter (Alt + Entrée)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# A exécuter (Alt + Entrée)
import pandas as pd
import altair as alt
from altair import *
import numpy as np
pd.set_option('display.float_format', lambda x: '%.4f' % x)

## Lecture de tables

3 tables SAP sont à votre disposition : 


*   [EKKO](https://raw.githubusercontent.com/hyqformat/formation_data_cacec/master/EKKO.csv) - table de stockage de l'en-tête des documents d'achat
*   [EKPO](https://raw.githubusercontent.com/hyqformat/formation_data_cacec/master/EKPO.csv) - table de stockage des items des documents d'achat
*   [KONV](https://raw.githubusercontent.com/hyqformat/formation_data_cacec/master/KONV.csv) - table de stockage des données transactionnelles


Ce sont des tables SAP 

In [0]:
## EXERCICE : Ecrire la commande Python qui permet de lire les 3 tables

# Indice : utiliser la librairie pandas et la fonction read_csv
ekko_table = # A FAIRE
ekpo_table = # A FAIRE
konv_table = # A FAIRE

In [0]:
## EXERCICE : Afficher les premières lignes d'une des tables (au choix)

In [0]:
## EXERCICE : Afficher les noms des colonnes des trois tables

Comme il y a trop de colonnes, nous choisissons de ne garder que certains attributs de chaque table : 

*   `EKKO` : `EBELN`, `ERNAM`, `LIFNR`, `KNUMV`
*   `EKPO` : `EBELN`, `EBELP`, `TXZ01`, `MATNR`, `MENGE`, `NETPR`, `NETWR`
*   `KONV` : `KNUMV`, `KPOSN`, `STUNR`, `KSCHL`, `KAWRT`, `KBETR`

---

Note : vous pouvez retrouver la signification de ces attributs aux liens suivants : 

*   [EKKO](https://www.se80.co.uk/saptables/e/ekko/ekko.htm)
*   [EKPO](https://www.leanx.eu/en/sap/table/ekpo.html)
*   [KONV](https://www.se80.co.uk/saptables/k/konv/konv.htm)




In [0]:
## EXERCICE : écrire trois listes avec les attributs à garder
kept_columns_ekko = # A FAIRE
kept_columns_ekpo = # A FAIRE
kept_columns_konv = # A FAIRE

In [0]:
## EXERCICE : filtrer les tables EKKO, EKPO et KONV à partir des trois vecteurs précédents
ekko = # A FAIRE
ekpo = # A FAIRE
konv = # A FAIRE

In [0]:
## EXERCICE : Visualiser les premières lignes d'une des trois tables

## Calculs sur les tables de commandes

On s'intéresse aux commandes du produit `BRENTCRUDE`, dont l'historique des données se trouve dans la table `EKPO`.

In [0]:
## EXERCICE : Filtrer la table EKPO en ne gardant que les observations dont l'attribut 'MATNR' est égal à 'BRENTCRUDE'
ekpo_bco = # A FAIRE

In [0]:
## EXERCICE : Combien de commandes pour le BRENTCRUDE ont été passées ? 

L'attribut **`NETWR`** correspond à la valeur nette du bien commandé. 

In [0]:
## EXERCICE : Calculer la valeur totale de Brent achetée

L'attribut **`MENGE`** correspond au volume de bien commandé.

In [0]:
## EXERCICE : Calculer le volume moyen commandé sur toutes les commandes de Brent

**Bonus** : Nous pouvons tracer un graphique pour jauger l'évolution des prix et des volumes selon la chronologie des ordres d'achat

In [0]:
## Prix du Brent en fonction de l'historique d'achat
v_netpr = alt.Chart().mark_line().encode(
  x = alt.X('index', axis = alt.Axis(title = "Ordre d'achat")),
  y = alt.Y('NETPR', scale = alt.Scale(domain = (4100, 4700)), axis = alt.Axis(title = "Prix net effectif de Brent Crude Oil"))
)
alt.layer(v_netpr, data = ekpo_bco.reset_index(), width = 600, height = 300)

In [0]:
v_menge = alt.Chart().mark_line().encode(
    x=alt.X('index', axis=alt.Axis(title="Ordre d'achat")),
    y=alt.Y('MENGE', scale=alt.Scale(domain=(19500, 20500)), axis=alt.Axis(title="Volume d'ordres de Brent"))
)
alt.layer(v_menge, data = ekpo_bco.reset_index(), width=600, height=300)

## Jointure entre les en-têtes et les commandes

`EKKO` et `EKPO` ont en commun d'avoir une colonne qui identifie le numéro de la commande (**`EBELN`**). 

In [0]:
## EXERCICE : Ecrire une jointure entre EKKO et EKPO (restreinte aux commandes de Brent) à partir de la colonne pivot `EBELN`

ekko_ekpo_bco = ## A FAIRE

## Visualiser les premières lignes du résultat

Trois attributs vont nous intéresser ici : 

*   **ERNAM** : identifiant de l'employé préposé aux achats.
*   **LIFNR** : identifiant du fournisseur de Brent.
*   **KNUMV** : identifiant des conditions associées à la transaction (utiles pour `KONV`).

In [0]:
## EXERCICE : retrouver le nombre de fournisseurs uniques, le nombre d'employés-acheteurs uniques et le nombre de conditions uniques

# indice : utiliser '.unique()' à la suite du filtrage de data frame

In [0]:
## EXERCICE : calculer le volume moyen d'ordres passés par chaque employé

# indice : utiliser 'groupby' sur la variable ERNAM et calculer la moyenne sur MENGE

**Bonus** : Nous pouvons retracer l'évolution des achats passés pour chaque employé. 

In [0]:
base = alt.Chart().mark_line().encode(
    x=alt.X('index', axis=alt.Axis(title="Ordres d'achat")),
    y=alt.Y('MENGE', scale=alt.Scale(domain=(19760, 20250)), axis=alt.Axis(title="Volume d'ordres d'achat passés sur le Brent")),
    color=alt.Color('ERNAM', legend=alt.Legend(title="Identifiant employé"))
)
alt.layer(base, data = ekko_ekpo_bco.reset_index(), width=600, height=300)

## Analyse de la table `KONV`

La table `KONV` est assez particulière car à un même identifiant de condition `KNUMV` est associé plusieurs éléments qui caractérisent les conditions dans lesquelles la transaction a été faite. 

In [0]:
## A exécuter
# On ne veut que les identifiants de conditions de transactions étant intervenus dans les commandes de Brent
konv_bco = konv[konv['KNUMV'].isin(ekko_ekpo_bco['KNUMV'])]

In [0]:
## EXERCICE : retrouver la liste des conditions uniques dans la colonne `KSCHL` de KONV

Explication du code des conditions : 
*   `PBXX` : Prix brut
*   `RB00` : Ristournes
*   `NAVM` : Déductions fiscales
*   `SKTO` : Ristournes en espèces
*   `WOTB` : Prix effectif

In [0]:
## EXERCICE : compter le nombre d'occurrences de chaque élément unique dans la colonne `KSCHL`

Que remarquez-vous ? 

In [0]:
# A exécuter : à chaque transaction, on associe une valeur binaire (0/1) pour savoir si oui ou non cette transaction a bénéficié d'une ristourne. 

konv_bco_d = konv_bco.join(konv_bco['KSCHL'].str.get_dummies())
rb00 = konv_bco_d.groupby('KNUMV')['RB00'].sum().reset_index()

In [0]:
# EXERCICE : réaliser la jointure entre ekko_ekpo_bco et rb00, à partir de la colonne KNUMV
eek_bco = # A FAIRE

## Détection de la fraude

In [0]:
## EXERCICE : Calculer le volume minimum et maximum des transactions pour lesquelles il y a eu un rabais

## EXERCICE : Calculer le volume minimum et maximum des transactions pour lesquelles il n'y a pas eu de rabais

Que constatez-vous ? 

In [0]:
# A exécuter
eek_bco.groupby('RB00')[['MENGE', 'ERNAM']].agg({'MENGE': 'mean', 'ERNAM': 'unique'})

Qui est le responsable de la fraude ? Pouvez-vous expliquer la fraude ? 

In [0]:
## EXERCICE : calculer le nombre de transactions concernées 

## Indice : faire la différence de longueur entre ekko_ekpo_bco et konv_bco[konv_bco['KSCHL']=='RB00']

In [0]:
## EXERCICE : calculer le montant moyen des transactions concernées

## Indice : faire la moyenne de konv_bco[konv_bco['KSCHL']=='RB00']['KBETR']

In [0]:
## EXERCICE : calculer le montant de la fraude

Chiffrez le sinistre ? 